# EDA by Youngseo Yoo
---

## Settings

In [2]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# get data
users = pd.read_csv('../data/users.csv')
books = pd.read_csv('../data/books.csv')
train = pd.read_csv('../data/train_ratings.csv')
test = pd.read_csv('../data/test_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('train shape: ', train.shape)
print('test shape: ', test.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
train shape:  (306795, 3)
test shape:  (76699, 3)


## Check users data

In [4]:
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   68092 non-null  int64  
 1   location  68092 non-null  object 
 2   age       40259 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [6]:
users.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,68092.0,139381.329539,80523.969862,8.0,69008.75,138845.5,209388.25,278854.0
age,40259.0,36.069873,13.842571,5.0,25.00,34.0,45.00,99.0


In [7]:
users['user_id'].nunique()

68092

In [8]:
users.isna().sum()

user_id         0
location        0
age         27833
dtype: int64

* Users의 age에 대한 결측치 비율이 상당히 높음

In [9]:
train_counts = train.groupby('user_id')['rating'].count().sort_values(ascending=False)
print(train_counts)
test_counts = test.groupby('user_id')['rating'].count().sort_values(ascending=False)
print(len(train_counts))
print(len(test_counts))

user_id
11676     5520
98391     4560
189835    1503
153662    1496
23902      956
          ... 
47317        1
150670       1
150676       1
150677       1
138957       1
Name: rating, Length: 59803, dtype: int64
59803
26167


* train 데이터 사용자는 약 6만명, test 데이터 사용자는 약 2.6만명

In [10]:
len(set(test['user_id']) - set(train['user_id']))

8266

* test의 2.6만명 중 0.8만명은 train에 없는 데이터
* cold start에 대한 대비가 필요

> rating 상위 10명 (train)

In [11]:
train_counts[:10]

/tmp/ipykernel_32038/2801245992.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_counts[:10]


user_id
11676     5520
98391     4560
189835    1503
153662    1496
23902      956
235105     812
76499      810
171118     771
16795      760
248718     747
Name: rating, dtype: int64

> rating 상위 10명 (test)

In [12]:
test_counts[:10]

/tmp/ipykernel_32038/1379518589.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  test_counts[:10]


user_id
11676     1419
98391     1125
189835     396
153662     347
23902      223
235105     208
76499      201
16795      198
248718     194
171118     189
Name: rating, dtype: int64

* train과 test는 단순히 분할되었음으로 유추

## Check books data

In [13]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg


In [14]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [15]:
books.describe().T

,count,mean,std,min,25%,50%,75%,max
year_of_publication,149570.0,1994.590606,8.179733,1376.0,1991.0,1996.0,2000.0,2006.0


In [16]:
books['isbn'].nunique()

149570

In [17]:
books['book_title'].nunique()

135436

* 동일한 제목의 책들이 일부 있음

In [18]:
books['book_author'].nunique()

62059

In [19]:
books['year_of_publication'].sort_values(ascending=True)

121860    1376.0
104259    1378.0
129205    1806.0
97563     1900.0
132721    1901.0
           ...  
55618     2005.0
92507     2005.0
104529    2005.0
79497     2005.0
81822     2006.0
Name: year_of_publication, Length: 149570, dtype: float64

In [20]:
books['publisher'].nunique()

11571

In [21]:
books['img_url'].nunique()

149570

In [22]:
books['img_path'].nunique()

149570

In [23]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
dtype: int64

* language, category, summary에 결측치가 많이 분포함

In [24]:
books[books['language'].isna() & books['summary'].notnull()]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


In [25]:
books[books['language'].notnull() & books['summary'].isna()]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


In [26]:
books[books['language'].isna() & books['category'].notnull()]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


In [27]:
books[books['language'].notnull() & books['category'].isna()][:3]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
138,2070567842,Folio Junior: L'histoire De Monsieur Sommer,Su>skind-Sempe,1994.0,Gallimard-Jeunesse,http://images.amazon.com/images/P/2070567842.0...,fr,NaN,Un petit garçon se pose énormément de question...,images/2070567842.01.THUMBZZZ.jpg
193,3596292646,Das Buch der lÃ?Â¤cherlichen Liebe.,Milan Kundera,2000.0,"Fischer (Tb.), Frankfurt",http://images.amazon.com/images/P/3596292646.0...,de,NaN,Die im &#39;Buch der lächerlichen Liebe&#39; v...,images/3596292646.01.THUMBZZZ.jpg
486,0871131811,Zodiac: The Eco-Thriller,Neal Stephenson,1988.0,Pub Group West,http://images.amazon.com/images/P/0871131811.0...,en,NaN,"Sangamon Taylor, an environmental extremist, s...",images/0871131811.01.THUMBZZZ.jpg


* language가 없으면 summary와 category는 없음
* language가 있어도 category가 없을 수 있음

## Check ratings data

In [28]:
train.head()

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9


In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  306795 non-null  int64 
 1   isbn     306795 non-null  object
 2   rating   306795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.0+ MB


In [30]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,306795.0,136128.416099,80512.194379,8.0,67591.0,134076.0,206438.0,278854.0
rating,306795.0,7.069714,2.433217,1.0,6.0,8.0,9.0,10.0


* train 기준, 평균 평점은 7.069714점

In [31]:
train.groupby('isbn')['rating'].count().sort_values(ascending=False)[:10]

isbn
0316666343    566
0971880107    465
0385504209    390
0312195516    307
0060928336    256
059035342X    251
0142001740    246
0446672211    236
044023722X    225
0452282152    223
Name: rating, dtype: int64

* 평가를 많이 받은 책

## Preprocess Users data

In [32]:
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


In [33]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.
# users.drop(columns='location', inplace=True)

/tmp/ipykernel_32038/2574908460.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [34]:
users.head()

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins,ontario,canada",NaN,timmins,ontario,canada
1,11400,"ottawa,ontario,canada",49.0,ottawa,ontario,canada
2,11676,"na,na,na",NaN,NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.0,toronto,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.0,victoria,britishcolumbia,canada


In [35]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       3254
location_country     2124
dtype: int64

In [36]:
users.shape

(68092, 6)

In [37]:
len(users[users['location_city'].isna() & users['location_state'].notnull()]['location_city'])

64

In [38]:
index = users[(users['location_city'].isna())&(users['location_state'].isna())&(users['location_country'].isna())].index
print(index)

Int64Index([2, 1679, 7760, 8634, 38772, 48206, 58435, 60682, 67104], dtype='int64')


* city, state, country 모두 없는 유저는 9명

In [39]:
users.drop(index, axis=0, inplace=True)
users.shape

(68083, 6)

In [40]:
users.isna().sum()

user_id                 0
location                0
age                 27826
location_city         113
location_state       3245
location_country     2115
dtype: int64

> country가 없는데 city가 있는 경우

In [41]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

users.isna().sum()

user_id                 0
location                0
age                 27826
location_city         113
location_state       3245
location_country      262
dtype: int64

> country가 없는데 state가 있는 경우

In [42]:
modify_location = users[(users['location_country'].isna())&(users['location_state'].notnull())]['location_state'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_state']==location.split(',')[1]].index,'location_country'] = location.split(',')[2]

users.isna().sum()

user_id                 0
location                0
age                 27826
location_city         113
location_state       3245
location_country      186
dtype: int64

> state가 없는데 city가 있는 경우

In [43]:
modify_location = users[(users['location_state'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]

users.isna().sum()

user_id                 0
location                0
age                 27826
location_city         113
location_state        610
location_country      186
dtype: int64

> state가 없는데 country가 있는 경우

In [44]:
# modify_location = users[(users['location_state'].isna())&(users['location_country'].notnull())]['location_country'].values

# location_list = []
# for location in modify_location:
#     try:
#         right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
#         location_list.append(right_location)
#     except:
#         pass

# for location in location_list:
#     # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
#     users.loc[users[users['location_country']==location.split(',')[2]].index,'location_state'] = location.split(',')[1]

# users.isna().sum()

> city가 없는데 country가 있는 경우

In [45]:
# modify_location = users[(users['location_city'].isna())&(users['location_country'].notnull())]['location_country'].values

# location_list = []
# for location in modify_location:
#     try:
#         right_location = users[(users['location'].str.contains(location))&(users['location_city'].notnull())]['location'].value_counts().index[0]
#         location_list.append(right_location)
#     except:
#         pass

# for location in location_list:
#     # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
#     users.loc[users[users['location_country']==location.split(',')[2]].index,'location_city'] = location.split(',')[0]

# users.isna().sum()

> city가 없는데 state가 있는 경우

In [46]:
# modify_location = users[(users['location_city'].isna())&(users['location_state'].notnull())]['location_state'].values

# location_list = []
# for location in modify_location:
#     try:
#         right_location = users[(users['location'].str.contains(location))&(users['location_city'].notnull())]['location'].value_counts().index[0]
#         location_list.append(right_location)
#     except:
#         pass

# for location in location_list:
#     # users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
#     users.loc[users[users['location_state']==location.split(',')[1]].index,'location_city'] = location.split(',')[0]

# users.isna().sum()

* 총 유저의 수는 약 6만 8천
* city의 결측치는 113에서 6으로 약 95% 감소
* state의 결측치는 3245에서 229로 약 93% 감소
* country의 결측치는 2115에서 186으로 약 91% 감소

In [47]:
# 결측치를 보유한 user 제거
index = users[(users['location_city'].isna()) | (users['location_state'].isna()) | (users['location_country'].isna())].index
print(index)
users.drop(index=index, inplace=True)
users.shape

Int64Index([  633,   919,  1008,  1012,  1065,  1200,  1291,  1334,  1442,
             1447,
            ...
            67314, 67402, 67530, 67805, 67854, 67889, 67973, 68007, 68075,
            68084],
           dtype='int64', length=693)


(67390, 6)

In [48]:
# users의 location 전처리 결과 확인
users.isna().sum()

user_id                 0
location                0
age                 27533
location_city           0
location_state          0
location_country        0
dtype: int64

In [49]:
# 전처리 결과를 바탕으로 location column 다시 채우기
users['location'].head(10)

0              timmins,ontario,canada
1               ottawa,ontario,canada
3              toronto,ontario,canada
4     victoria,britishcolumbia,canada
5               ottawa,ontario,canada
6                            ottawa,,
7             kingston,ontario,canada
8               ottawa,ontario,canada
9               comber,ontario,canada
10              guelph,ontario,canada
Name: location, dtype: object

In [50]:
users['location'] = users['location_city'] + "," + users['location_state'] + "," + users['location_country']

In [51]:
users['location'].head(10)

0              timmins,ontario,canada
1               ottawa,ontario,canada
3              toronto,ontario,canada
4     victoria,britishcolumbia,canada
5               ottawa,ontario,canada
6               ottawa,ontario,canada
7             kingston,ontario,canada
8               ottawa,ontario,canada
9               comber,ontario,canada
10              guelph,ontario,canada
Name: location, dtype: object

In [58]:
users.drop(['location_city', 'location_state', 'location_country'], axis=1, inplace=True)
users.shape

(67390, 3)

In [61]:
users.head(10)

,user_id,location,age
0,8,"timmins,ontario,canada",NaN
1,11400,"ottawa,ontario,canada",49.0
3,67544,"toronto,ontario,canada",30.0
4,85526,"victoria,britishcolumbia,canada",36.0
5,96054,"ottawa,ontario,canada",29.0
6,116866,"ottawa,ontario,canada",NaN
7,123629,"kingston,ontario,canada",NaN
8,177458,"ottawa,ontario,canada",29.0
9,200273,"comber,ontario,canada",NaN
10,210926,"guelph,ontario,canada",NaN


In [60]:
# make csv from df
users.to_csv('users.csv', index=False)

test